In [1]:
import numpy as np
import pandas as pd
import cobra.test
import os

from os.path import join

from time import time


from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

a = cobra.io.read_sbml_model("iEC1344_C.xml")

import cobra
from cobra.test import create_test_model
cobra_config = cobra.Configuration()
cobra_config.solver = "gurobi"
model = create_test_model("textbook")
model.solver

Academic license - for non-commercial use only - expires 2023-02-21
Using license file /home/schwartzlab/gurobi.lic


In [2]:
a.reactions.EX_o2_e.upper_bound= a.reactions.EX_o2_e.lower_bound=0

In [3]:
a.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.001257,0,0.00%
cl_e,EX_cl_e,0.001257,0,0.00%
co2_e,EX_co2_e,0.08881,1,0.15%
cobalt2_e,EX_cobalt2_e,6.038E-06,0,0.00%
cu2_e,EX_cu2_e,0.0001712,0,0.00%
fe2_e,EX_fe2_e,0.001993,0,0.00%
fe3_e,EX_fe3_e,0.001886,0,0.00%
glc__D_e,EX_glc__D_e,10,6,99.85%
h2o_e,EX_h2o_e,1.709,0,0.00%
k_e,EX_k_e,0.04714,0,0.00%


In [4]:
solution = a.optimize()
print(solution)

solution.objective_value

<Solution 0.242 at 0x7fcd1879a070>


0.2415061772036904

In [4]:
fmin = -1000
fmax = 1000
i = -1

while i<2797:
    x = list()    # x and y need to be reinitialised for each new reaction
    y = list()
    f = fmin      # define a new variable to change the flux, fmin should remain a fixed value

    i = i + 1
    r = a.reactions[i]
    r_id = r.id

    foriginalmax = a.reactions.get_by_id(r_id).upper_bound
    foriginalmin = a.reactions.get_by_id(r_id).lower_bound
    a.reactions.get_by_id(r_id).lower_bound = a.reactions.get_by_id(r_id).upper_bound = f

    while f <= fmax:
        a.reactions.get_by_id(r_id).lower_bound = a.reactions.get_by_id(r_id).upper_bound = f
        
        solution = a.optimize()
        ov_new = solution.objective_value

        x.append(f)
        y.append(ov_new)

        f = f + 10

    d = pd.DataFrame(x, columns = ['flux'])
    d2 = pd.DataFrame(y, columns = ['Biomass'])
    d['Biomass'] = d2['Biomass']
    biomass_per_flux = d.to_csv('biomass_per_flux_for_reaction_{}_anaerobic.csv'.format(r_id), index = True)
    a.reactions.get_by_id(r_id).lower_bound = foriginalmin
    a.reactions.get_by_id(r_id).upper_bound = foriginalmax

/home/schwartzlab/miniconda3/lib/python3.8/site-packages/cobra/util/solver.py:508: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


IndexError: list index out of range

In [6]:
len(a.reactions)

2726

In [5]:
solution.fluxes
fluxes_after_FBA = solution.fluxes.to_csv('fluxes_after_FBA_anaerobic.csv', index = True)

In [34]:
a.reactions.get_by_id("XYLI1").lower_bound = a.reactions.get_by_id("XYLI1").upper_bound = 0

solution = a.optimize()
print(solution)

solution.objective_value


<Solution 0.242 at 0x7ff08606feb0>


0.24150617720368944